### Import Modules

In [13]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [14]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [24]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        'include_log' : False, # alphaS
        'include_log2' : False
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFK', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    'bias_correct' : True,
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

In [25]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'], p_dict['F2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'],)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
elif p_dict['save_prior']:
    print 'Saving prior...'
    new_prior = bootstrapper.create_prior_from_fit()
    print new_prior
    data_loader.save_prior(new_prior, p_dict['fit_type'], p_dict['F2'])
else:
    print bootstrapper
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.430999994278
Compiling results...


Fit type: ma (F^2 = FKFK, bsN = 1)

Fitting to nnlo 
 with volume corrections O(0) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1756(18) [1.1932(19)]   (delta_su2 = -0.00234(18))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      L_5 0            0.00031 (62)        A_a 0               0.74 (54)
          1             0.0000 (50)            1               0.0 (5.0)
      A_k 0               0.2 (3.1)        A_p 0               0.2 (1.0)
          1               0.0 (5.0)            1               0.0 (5.0)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 2.1 [16]    Q = 0.0057    logGBF = 54.696

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 11/0.3)
  fitter = scipy_least

In [27]:
def _make_fitter(self, j):
    prepped_data = self._make_fit_data(j)
    # Need to randomize prior in bayesian-bootstrap hybrid
    temp_prior = self._randomize_prior(self.prior, j)
    temp_fitter = fit.fitter(fit_data=prepped_data, prior=temp_prior,
                    order=self.order, fit_type=self.fit_type, chain_fits=self.chain_fits)
    return temp_fitter

temp_fitter = _make_fitter(bootstrapper, 0)

(0.1, 0.1, 0.1)
(0.1, 0.1, 1.0)
(0.1, 0.1, 10.0)
(0.1, 1.0, 1.0)
(0.1, 1.0, 10.0)
(0.1, 10.0, 10.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 10.0)
(1.0, 10.0, 10.0)
(10.0, 10.0, 10.0)


In [66]:
np.exp(np.linspace(np.log(0.1), np.log(10.0), 10))

array([ 0.1       ,  0.16681005,  0.27825594,  0.46415888,  0.77426368,
        1.29154967,  2.15443469,  3.59381366,  5.9948425 , 10.        ])

In [81]:
from itertools import combinations_with_replacement  

def grid_search(self):
    models = self._make_models()
    y_data = self._make_y_data()
    prior = self._make_prior()
    
    prior_lecs = {
        'L_5' : '0(1)',
        'A_a' : '0(1)',
        'A_k' : '0(1)',
        'A_p' : '0(1)',
    }
    prior_lecs = gv.gvar(prior_lecs)
    fitter = lsqfit.MultiFitter(models=models)
    
    
    zs = np.exp(np.linspace(np.log(0.1), np.log(10.0), 6))
    combs = combinations_with_replacement (zs, len(prior_lecs))
    newprior = {key : prior[key] for key in prior.keys()}
    
    fits_array = []
    for l, comb in enumerate(combs):
        print l
        for j, key in enumerate(prior_lecs.keys()):
            newprior[key] = comb[j] *prior_lecs[key]
        
        fit = fitter.lsqfit(data=y_data, prior=newprior, fast=False)
        fits_array.append(fit)
        
    return fits_array
    
    #prior = {self._make_prior().keys()}
    
t0 = time.time()
fits = grid_search(temp_fitter)
t1 = time.time()
print "Time: ", t1 - t0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
Time:  65.3199999332


In [82]:
arg = np.argmax([fit.logGBF for fit in fits])
print fits[arg]

Least Square Fit:
  chi2/dof [dof] = 2.5 [16]    Q = 0.001    logGBF = 54.133

Parameters:
     lam2_chi 0      0.3799 (11)       [    0.3791 (11) ]  
              1      0.4042 (11)       [    0.4034 (13) ]  
              2      0.4153 (10)       [    0.4151 (11) ]  
              3      0.4317 (11)       [    0.4317 (11) ]  
              4      0.6916 (22)       [    0.6933 (23) ]  
              5      0.6971 (22)       [    0.6956 (23) ]  
              6      0.7008 (27)       [    0.6951 (36) ]  *
              7      0.7026 (23)       [    0.7033 (24) ]  
              8      0.7254 (18)       [    0.7244 (20) ]  
              9      0.7424 (15)       [    0.7412 (16) ]  
             10      0.7694 (15)       [    0.7691 (15) ]  
             11      1.0786 (38)       [    1.0812 (41) ]  
             12      1.0620 (23)       [    1.0610 (26) ]  
             13      1.0844 (21)       [    1.0865 (23) ]  
             14      1.1067 (22)       [    1.1064 (22) ]  
        

In [121]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'], p_dict['F2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'],)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
elif p_dict['save_prior']:
    print 'Saving prior...'
    new_prior = bootstrapper.create_prior_from_fit()
    print new_prior
    data_loader.save_prior(new_prior, p_dict['fit_type'], p_dict['F2'])
else:
    print bootstrapper
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
Done!
100% complete Time (s):  3.77700018883
Compiling results...


Fit type: ma-ratio (F^2 = FKFK, bsN = 1)

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1832(20) [1.1932(19)]   (delta_su2 = -0.00255(19))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      L_4 0            0.00010 (45)     A_loga 0             4(9800)e-07
          1             0.0000 (10)            1            0.00000 (98)
      L_5 0            0.00027 (27)        A_a 0             6(9800)e-07
          1             0.0000 (10)            1            0.00000 (98)
      A_p 0               0.19 (40)        A_k 0               0.45 (69)
          1               0.00 (88)            1               0.00 (88)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1.7 [16]    Q = 0.0

### Do fit

In [119]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'], p_dict['F2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'],)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
elif p_dict['save_prior']:
    print 'Saving prior...'
    new_prior = bootstrapper.create_prior_from_fit()
    print new_prior
    data_loader.save_prior(new_prior, p_dict['fit_type'], p_dict['F2'])
else:
    print bootstrapper
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  10.5600001812
Compiling results...


Fit type: ma-ratio (F^2 = FKFK, bsN = 1)

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1811(43) [1.1932(19)]   (delta_su2 = -0.00249(22))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      L_4 0           -0.00412 (27)     A_loga 0               4.1 (4.2)
          1             0.0000 (50)            1               0.0 (5.0)
      L_5 0           -0.00066 (21)        A_a 0               7.0 (3.2)
          1             0.0000 (50)            1               0.0 (5.0)
      A_p 0               0.9 (1.2)        A_k 0              -4.1 (2.1)
          1               0.0 (5.0)            1               0.0 (5.0)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] 

In [102]:
    def fk_fpi_fit_fcn(self, fit_data=None, fit_parameters=None, fit_type=None):
        if fit_data is None:
            fit_data = self.get_phys_point_data()
        if fit_parameters is None:
            fit_parameters = self.get_fit_parameters().copy()
        if fit_type is None:
            fit_type = 'xpt'

        for key in fit_data.keys():
            fit_parameters[key] = fit_data[key]
            
        temp_prior = self._randomize_prior(self.prior, j)
        
        temp_fitter = fit.fitter(fit_data=fit_data, prior=temp_prior, 
                                 order=self.order, fit_type=fit_type, chain_fits=self.chain_fits)
        fitfcn = temp_fitter._make_models()[0].fitfcn
        
        print fitfcn(p=fit_parameters)

        #return temp_fit.fcn(p=fit_parameters)[model_name]
    
    
    fk_fpi_fit_fcn(bootstrapper)

1.1851(37)


### Make all fits

In [105]:
p_dict = {
    'order' : {
        'fit' : 'nnlo'
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'save_results' : True,
}

In [107]:
#results_dict = {}

data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

for fit_type in ['ma', 'xpt', 'ma-ratio', 'xpt-ratio']:#, 'ma-old']:
    for fit_order in ['nlo', 'nnlo']:
        #results_dict[fit_type] = {}
        for include_log2 in [True, False]:
            #results_dict[fit_type][include_log2] = {}
            for include_log in [True, False]:
                #results_dict[fit_type][include_log2][include_log] = {}
                for vol in [0, 10]:
                    #results_dict[fit_type][include_log2][include_log][vol] = {}
                    #print fit_type, order_fit

                    t0 = time.time()

                    # Specify fit parameters
                    p_dict['order']['include_log'] = include_log
                    p_dict['order']['include_log2'] = include_log2
                    p_dict['order']['vol'] = vol
                    p_dict['order']['fit'] = fit_order
                    p_dict['fit_type'] = fit_type

                    # Do fit

                    # Load data
                    data_loader = dl.data_loader()
                    fit_data = data_loader.get_fit_data()

                    # Get prior
                    prior = None
                    if p_dict['use_prior']:
                        prior = data_loader.get_prior(p_dict['fit_type'])

                    # Make bootstrapper
                    bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                                   fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])


                    #results_dict[fit_type][include_log2][include_log][vol]['fit'] = bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)
                    #results_dict[fit_type][include_log2][include_log][vol]['L5'] = bootstrapper.get_fit_parameters('L_5')
                    #results_dict[fit_type][include_log2][include_log][vol]['info'] = bootstrapper.get_fit_info()
                    #results_dict[fit_type][include_log2][include_log][vol]['fit']
                    
                    if p_dict['save_results']:
                        data_loader.save_fit_info(bootstrapper.get_fit_info())
                    
                    t1 = time.time()

                    #print test(bootstrapper)
                    
                    print bootstrapper.get_fit_info()['name']
                    print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.493999958038
Compiling results...
Saving...
Done.
ma_nlo_alphaS_logSq

Total time (s):  1.15199995041 

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.13100004196
Compiling results...
Saving...
Done.
ma_nlo_FV_alphaS_logSq

Total time (s):  2.3220000267 

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.401000022888
Compiling results...
Saving...
Done.
ma_nlo_logSq

Total time (s):  1.0490000248 

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.03900003433
Compiling results...
Saving...
Done.
ma_nlo_FV_logSq

Total time (s):  2.20799994469 

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.44000005722
Compiling results...
Saving...
Done.
ma_nlo_alphaS

Total time (s):  1.08400011063 

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.15599989891
Compiling results...
Saving...
Done.
ma_nlo_FV_alphaS

Total time (s):  2

Done!
100% complete Time (s):  1.27400016785
Compiling results...
Saving...
Done.
xpt-ratio_nlo_alphaS_logSq

Total time (s):  1.88400006294 

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.867000103
Compiling results...
Saving...
Done.
xpt-ratio_nlo_FV_alphaS_logSq

Total time (s):  2.56200003624 

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.996999979019
Compiling results...
Saving...
Done.
xpt-ratio_nlo_logSq

Total time (s):  1.61199998856 

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.32299995422
Compiling results...
Saving...
Done.
xpt-ratio_nlo_FV_logSq

Total time (s):  2.00200009346 

Using default prior.
Making fits...
Done!
100% complete Time (s):  3.81699991226
Compiling results...
Saving...
Done.
xpt-ratio_nlo_alphaS

Total time (s):  4.4399998188 

Using default prior.
Making fits...
Done!
100% complete Time (s):  3.14800000191
Compiling results...
Saving...
Done.
xpt-ratio_nlo_FV_alphaS

Total time (s)

In [18]:
translator_dict = {
    'xpt' : 'xpt-ratio', 
    'xpt-taylor' : 'xpt',
    'ma' : 'ma-ratio', 
    'ma-taylor' : 'ma',
}

#print "name   FK/Fpi   L5   k   p   s"

k = 30 - len("name")
output = "name" +k*' '

for param in ['FK/Fpi', 'L5', 'k', 'p', 's']:
    k = 14 - len(param)
    output += k*" " + str(param)

print output
                        
                        
print "---"

#for fit_type in results_dict.keys():
    #for include_log2 in results_dict[fit_type].keys():
for include_log2 in results_dict['ma'].keys():
    for include_log in results_dict[fit_type][include_log2].keys():
        for vol in results_dict[fit_type][include_log2][include_log].keys():
            for fit_type in ['xpt-taylor', 'ma-taylor']:
                
                name = translator_dict[fit_type]
                if vol > 0:
                    name = name + '_FV'
                    
                if include_log:
                    name = name + '_alphaS'
                
                if include_log2:
                    name = name + '_logSq'
                    
                    
                fit = results_dict[fit_type][include_log2][include_log][vol]['fit']
                L5 = results_dict[fit_type][include_log2][include_log][vol]['info']['L_5']
                Ak = results_dict[fit_type][include_log2][include_log][vol]['info']['A_k']
                Ap = results_dict[fit_type][include_log2][include_log][vol]['info']['A_p']
                Aa = results_dict[fit_type][include_log2][include_log][vol]['info']['A_a']
                
                if ((vol==0), include_log, include_log2) in [(1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]:
                    if len(name) > 0:
                        k = 30 - len(name)
                        output = name +k*' '
                        
                        for param in [fit, L5, Ak, Ap, Aa]:
                            k = 14 - len(str(param))
                            output += k*" " + str(param)
                            
                        print output
                
                    


name                                  FK/Fpi            L5             k             p             s
---
xpt                               1.1811(25)  -0.00035(49)      2.3(2.1)      1.59(39)     -3.32(36)
ma                                1.1812(23)   0.00041(57)     -0.8(2.4)      1.18(44)      1.17(40)
xpt_FV                            1.1821(27)  -0.00059(49)      3.3(2.1)      1.36(39)     -3.85(36)
ma_FV                             1.1811(23)   0.00034(57)     -0.5(2.4)      1.11(44)      0.96(40)
xpt_alphaS                        1.1806(38)  -0.00037(50)      2.3(2.1)      1.59(39)     -2.9(2.8)
ma_alphaS                         1.1820(36)   0.00043(57)     -0.9(2.4)      1.19(44)      0.4(2.8)
xpt_alphaS_logSq                  1.1832(36)  -0.00032(50)      0.1(2.2)      2.24(40)     -2.9(2.8)
ma_alphaS_logSq                   1.1855(36)   0.00006(58)     -1.3(2.4)      1.53(44)      0.4(2.8)
